In [1]:
import datetime
import math
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import string

In [2]:
borough_to_cluster = pd.read_csv("./data/borough_to_cluster.csv")

In [3]:
borough_to_cluster.head()

,LOCATION,Cluster
0,Barking and Dagenham,1
1,Barnet,1
2,Bexley,1
3,Brent,2
4,Bromley,1


In [4]:
borough_to_cluster.LOCATION

0       Barking and Dagenham
1                     Barnet
2                     Bexley
3                      Brent
4                    Bromley
5                     Camden
6             City of London
7                    Croydon
8                     Ealing
9                    Enfield
10                 Greenwich
11                   Hackney
12    Hammersmith and Fulham
13                  Haringey
14                    Harrow
15                  Havering
16                Hillingdon
17                  Hounslow
18                 Islington
19    Kensington and Chelsea
20      Kingston upon Thames
21                   Lambeth
22                  Lewisham
23                    Merton
24                    Newham
25                 Redbridge
26      Richmond upon Thames
27                 Southwark
28                    Sutton
29             Tower Hamlets
30            Waltham Forest
31                Wandsworth
32               Westminster
Name: LOCATION, dtype: object

In [8]:
demographics = pd.read_csv("./data/london_borough_demographics.csv")

In [9]:
demographics.head()

,area_name,inner_outer_london,prop_aged_0_15,prop_working_age,prop_aged_over_65,prop_residents_born_abroad,unemployment_rate,gross_annual_pay,crime_rate_per_thousand,median_house_price,home_owned_outright,home_being_bought_with_mortgage,turnout_local_elections
0,Barking and Dagenham,Outer London,27.2,63.1,9.7,37.8,11.0,27886,83.4,243500,16.4,27.4,36.5
1,Barnet,Outer London,21.1,64.9,14.0,35.2,8.5,33443,62.7,445000,32.4,25.2,40.5
2,Bexley,Outer London,20.6,62.9,16.6,16.1,7.6,34350,51.8,275000,38.1,35.3,39.6
3,Brent,Outer London,20.9,67.8,11.3,53.9,7.5,29812,78.8,407250,22.2,22.6,36.3
4,Bromley,Outer London,19.9,62.6,17.5,18.3,5.3,37682,64.1,374975,37.8,34.9,40.8


In [27]:
data = demographics.merge(borough_to_cluster, left_on='area_name', right_on='LOCATION', how='inner')

In [33]:
data.head()

,area_name,inner_outer_london,prop_aged_0_15,prop_working_age,prop_aged_over_65,prop_residents_born_abroad,unemployment_rate,gross_annual_pay,crime_rate_per_thousand,median_house_price,home_owned_outright,home_being_bought_with_mortgage,turnout_local_elections,LOCATION,Cluster
0,Barking and Dagenham,Outer London,27.2,63.1,9.7,37.8,11.0,27886,83.4,243500,16.4,27.4,36.5,Barking and Dagenham,1
1,Barnet,Outer London,21.1,64.9,14.0,35.2,8.5,33443,62.7,445000,32.4,25.2,40.5,Barnet,1
2,Bexley,Outer London,20.6,62.9,16.6,16.1,7.6,34350,51.8,275000,38.1,35.3,39.6,Bexley,1
3,Brent,Outer London,20.9,67.8,11.3,53.9,7.5,29812,78.8,407250,22.2,22.6,36.3,Brent,2
4,Bromley,Outer London,19.9,62.6,17.5,18.3,5.3,37682,64.1,374975,37.8,34.9,40.8,Bromley,1


In [11]:
from sklearn.svm import LinearSVC

In [28]:
# 0 Redbridge
# 1 Newham
# 2 Southwark
# 3 Westminster
train = data.drop([23,24,26,31])
test = data.iloc[[23,24,26,31],:]

In [102]:
clf = LinearSVC(random_state=0, tol=1e-5, C=0.001)

In [103]:
X_train, y_train = train.drop(['Cluster'], axis=1), train[["Cluster"]]

In [104]:
X_test, y_test = test.drop(['Cluster'], axis=1), test[["Cluster"]]

In [105]:
X_train.head()

,area_name,inner_outer_london,prop_aged_0_15,prop_working_age,prop_aged_over_65,prop_residents_born_abroad,unemployment_rate,gross_annual_pay,crime_rate_per_thousand,median_house_price,home_owned_outright,home_being_bought_with_mortgage,turnout_local_elections,LOCATION
0,Barking and Dagenham,Outer London,27.2,63.1,9.7,37.8,11.0,27886,83.4,243500,16.4,27.4,36.5,Barking and Dagenham
1,Barnet,Outer London,21.1,64.9,14.0,35.2,8.5,33443,62.7,445000,32.4,25.2,40.5,Barnet
2,Bexley,Outer London,20.6,62.9,16.6,16.1,7.6,34350,51.8,275000,38.1,35.3,39.6,Bexley
3,Brent,Outer London,20.9,67.8,11.3,53.9,7.5,29812,78.8,407250,22.2,22.6,36.3,Brent
4,Bromley,Outer London,19.9,62.6,17.5,18.3,5.3,37682,64.1,374975,37.8,34.9,40.8,Bromley


In [106]:
y_train.head()

,Cluster
0,1
1,1
2,1
3,2
4,1


In [110]:
# columns = ["prop_aged_0_15", "prop_working_age", "prop_aged_over_65", "prop_residents_born_abroad", "unemployment_rate", "crime_rate_per_thousand", "median_house_price", "home_owned_outright", "home_being_bought_with_mortgage", "turnout_local_elections"]
# columns = ["prop_aged_over_65"]
# columns = ["home_owned_outright"]
# columns = ["home_being_bought_with_mortgage"]
columns = ["prop_aged_over_65", "home_owned_outright", "home_being_bought_with_mortgage"]

clf.fit(X_train[columns],y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearSVC(C=0.001, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)

In [111]:
clf.predict(X_test[columns])

array([1, 1, 2, 1])

In [112]:
y_test

,Cluster
23,1
24,0
26,2
31,3
